In [59]:
import pickle
import sys
sys.path.append('..')
with open('../data/tt_subset.pkl','rb') as file:
    data = pickle.load(file)

In [60]:
X, y = data['graph'], data['tt']

In [11]:
from app.kernel import *
from app.smiles import *
from app.ActiveLearning import *
from app.Nystrom import NystromGaussianProcessRegressor
kernel_config = KernelConfig(save_mem=False, property='tt')

In [22]:
from sklearn.gaussian_process import GaussianProcessRegressor
model = GaussianProcessRegressor(kernel=kernel_config.kernel, random_state=0, optimizer=None,
                                            normalize_y=True, alpha=0.01).fit(X[:4], y[:4])

In [39]:
def predict_manual_loocv(i):
    train_X = np.delete(np.array(X), i)
    train_y = np.delete(np.array(y), i)
    model = GaussianProcessRegressor(kernel=kernel_config.kernel, random_state=0, optimizer=None,
                                            normalize_y=True, alpha=0.01).fit(train_X, train_y)
    y_pred, y_std = model.predict(X, return_std=True)
    return y_pred[i], y_std[i]

In [32]:
import scipy
K = model.kernel_(X)
K[np.diag_indices_from(K)] += 0.01
I_mat = np.eye(K.shape[0])
K_inv = scipy.linalg.cho_solve(scipy.linalg.cho_factor(K,lower=True), I_mat)
#K_inv = np.linalg.inv(K) 
y_pred = y - K_inv.dot(y) / K_inv.diagonal()
y_std = np.sqrt(1/ K_inv.diagonal())

In [33]:
# using loocv formula
y_pred, y_std

(0    217.296661
 1    224.961172
 2    229.758814
 3    145.204327
 4    236.462419
 Name: tt, dtype: float64,
 array([0.14255205, 0.14698835, 0.12322945, 0.55479011, 0.55450834]))

In [42]:
# using manual loop
manual_y = np.array([predict_manual_loocv(i) for i in range(5)])

In [57]:
df = pd.DataFrame({'Smiles':list(map(lambda x:x.smiles, X)), 'y':y, 'y_pred_loocv':y_pred, 'y_pred_loop':manual_y[:,0],'y_std_loocv':y_std, 'y_std_loop':manual_y[:,1]})

In [58]:
df

,Smiles,y,y_pred_loocv,y_pred_loop,y_std_loocv,y_std_loop
0,CCCCCCCCCC,243.536,217.296661,219.139957,0.142552,0.101593
1,CCCCCCCC,216.418,224.961172,226.670629,0.146988,0.107729
2,CCCCCCCCC,219.680,229.758814,230.576508,0.123229,0.072010
3,c1ccccc1,278.674,145.204327,185.644210,0.554790,0.545703
4,Cc1ccccc1,178.188,236.462419,269.730480,0.554508,0.545417
